In [46]:
!pip install streamlit wikipedia-api sentence-transformers pyngrok

In [47]:
import pandas as pd

# Load supply chain risk dataset
df = pd.read_csv("/content/supply_chain_data.csv")  # Modify with your dataset path

# Check dataset structure
print(df.head())

# Convert text column into a list
documents = df["Product type"].tolist()

  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626               Unknown             1          10   
3        7766.836426            Non-binary            23          13   
4        2686.505152            Non-binary             5           3   

   Order quantities  ...  Location Lead time  Production volumes  \
0                96  ...    Mumbai        29          

In [48]:
!pip install faiss-cpu

In [49]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a pre-trained sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text data into dense vector embeddings
embeddings = model.encode(documents, convert_to_numpy=True)

# Build a FAISS index for fast similarity search
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance for similarity search
index.add(embeddings)  # Add embeddings to the FAISS index

# Save index and embeddings for future use
faiss.write_index(index, "supply.faiss")
np.save("supply_embeddings.npy", embeddings)

In [63]:
def retrieve_all_products(query):
    """
    Retrieve all relevant products based on a user query.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, len(df))  # Retrieve all possible results
    results = list(dict.fromkeys([df["Product type"].iloc[i] for i in indices[0]]))  # Remove duplicates
    return results

# Example Usage
query = "What are the major logistics risks this quarter?"
results = retrieve_all_products(query)
for res in results:
    print(res)


cosmetics
haircare
skincare


In [64]:
import streamlit as st
import wikipediaapi
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
import os
import pandas as pd

# Function to fetch Wikipedia summary
def fetch_wikipedia_summary(query):
    wiki = wikipediaapi.Wikipedia(language='en', user_agent="Mozilla/5.0 (compatible; MyBot/0.1; +http://mybot.com/info)")
    page = wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No Wikipedia page found for the given query."


In [59]:
# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [74]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# Load FAISS index and dataset
df = pd.read_csv("/content/supply_chain_data.csv")  # Modify dataset path
documents = df["Product type"].tolist()

# Load pre-trained model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index and embeddings
index = faiss.read_index("supply.faiss")

# Streamlit UI
st.title("Supply Chain Product Analysis")
st.write("Enter a query to retrieve the most relevant products.")

query = st.text_input("Enter your query:")
if st.button("Search Products"):
    if query:
        query_embedding = model.encode([query], convert_to_numpy=True)
        _, indices = index.search(query_embedding, 3)  # Retrieve all possible results
        results = list(dict.fromkeys([df["Product type"].iloc[i] for i in indices[0]]))  # Remove duplicates

        st.subheader("🔍 Top Products:")
        for i, res in enumerate(results):
            st.write(f"*{i+1}.* {res}")

    else:
        st.warning("Please enter a valid query.")

Overwriting app.py


In [75]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.245.108.172:8502

  Stopping...


In [76]:
# Kill any existing ngrok and Streamlit processes
!pkill -f streamlit
!pkill -f ngrok

# Re-run ngrok authentication
!ngrok authtoken 2suy40kBbJbJZebBZ5sYNAQFc39_3bNiSzwj1aZ7u1R3JZjKw

# Start Streamlit again
!nohup streamlit run app.py --server.port 8501 &

# Reconnect ngrok to expose Streamlit
from pyngrok import ngrok
import time

time.sleep(5)  # Wait for Streamlit to start
public_url = ngrok.connect(8501)
print(f"🚀 New Public Link: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
nohup: appending output to 'nohup.out'
🚀 New Public Link: NgrokTunnel: "https://f009-35-245-108-172.ngrok-free.app" -> "http://localhost:8501"
